In [1]:
import zarr
import xarray as xr
import time
import dask

In [2]:
ask_workers=2

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = SLURMCluster(cores=1, memory='1.5TB',job_extra=['--partition=prepost','--account=yrf@cpu',
                                  '--ntasks=1','--cpus-per-task=1'], project='PerfTestPangeo', walltime='02:30:00')
cluster.scale(ask_workers)

c = Client(cluster)

c

Client Scheduler: tcp://10.159.17.19:37273 Dashboard: http://10.159.17.19:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
!squeue -u rote001
from dask.utils import ensure_dict, format_bytes
workers = c.scheduler_info()["workers"]
text="Workers= " + str(len(workers))
memory = [w["memory_limit"] for w in workers.values()]
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           2128791   prepost dask-wor  rote001 PD       0:00      1 (Priority)
           2128792   prepost dask-wor  rote001 PD       0:00      1 (Resources)
           2128770    cpu_p1     srun  rote001  R       3:17      1 r5i7n9
Workers= 0, Memory=0 B


In [41]:
%time ds=xr.open_zarr('/gpfswork/rech/yrf/rote001/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

CPU times: user 145 ms, sys: 22.6 ms, total: 167 ms
Wall time: 165 ms


In [42]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 101 ms, sys: 6.5 ms, total: 108 ms
Wall time: 126 ms


In [ ]:
%time mean.load()